# Counting the route step coordinates

This script is to count the all OSRM route step coordinates from the training data set. The training data set has 1.4 million trips as each trip has multiple step coordinates, the output CSV will count more than 2 million coordinates. For instance, the output CSV file shows the maximum counting number is 43763 and its coordinate is -73.973249, 40.739237, which means there are 43763 trip routes from the training data that pass this coordinate. 


In [ ]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections

In [2]:
conf = SparkConf().setMaster("local").setAppName("Coordinates")
sc = SparkContext(conf = conf)

In [10]:
lines = sc.textFile("train_full_parsed_clean3.1.csv")

header = lines.first()

header1= sc.parallelize([header])
linesnoheader = lines.subtract(header1)

In [11]:
def loaddirection(line):
    
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    return coordinates

In [12]:
direction = lines.flatMap(loaddirection)
direction

PythonRDD[26] at RDD at PythonRDD.scala:48

In [13]:
wordCounts = direction.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey(False)
results = wordCountsSorted.collect()

In [14]:
df=pd.DataFrame(results,columns=['count','coordinates'])

In [15]:
df['longitude']=df['coordinates'].apply(lambda x: re.findall('(-?\d+\.\d+)',x)[0])
df['latitude']=df['coordinates'].apply(lambda x: re.findall('(-?\d+\.\d+)',x)[1])

In [16]:
df[['count','longitude','latitude']].to_csv("newcounter.csv",index=False)